In [14]:
#importing the libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle as pkl
from tensorflow.keras.models import load_model
import dill

In [15]:
#loading all models
with open("ohe.pkl","rb") as file:
    ohe = pkl.load(file)
with open("sc.pkl","rb") as file:
    sc = pkl.load(file)
with open("gender_encode.dill","rb") as file:
    ge = dill.load(file)

model = load_model("ann.h5")
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)



In [16]:
#example data point
idata = {
   'CreditScore':600,
   'Geography':'France',
   'Gender':'Male',
   'Age':40,
   'Tenure':3,
   'Balance':60000 ,
   'NumOfProducts':2 ,
   'HasCrCard':1,
   'IsActiveMember':1,
   'EstimatedSalary':50000
}

df1 = pd.DataFrame([idata.values()], columns=idata.keys())
df1.head()

def preprocess(id):
    id["Gender"] = ge(id)
    id1  = pd.DataFrame(ohe.transform(id["Geography"].values.reshape(-1,1)).toarray(),columns=ohe.get_feature_names_out(['Geography']))
    id.drop("Geography",inplace=True,axis=1)
    id = pd.concat([id,id1],axis=1)
    id = sc.transform(id)
    return id

df1 = preprocess(df1)
df1

array([[-0.52111599,  0.91601335,  0.10961719, -0.68538967, -0.2569057 ,
         0.8095029 ,  0.64259497,  0.9687384 , -0.87203322,  0.98560362,
        -0.5698444 , -0.57369368]])

In [17]:
idata.keys()

dict_keys(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary'])

In [18]:
#predict churn
pred = model.predict(df1)
pred

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step


array([[0.03741987]], dtype=float32)

In [19]:
if pred[0][0]>0.5:
    print("churn")
else:
    print("not churn")

not churn
